# 1. Download SRA data

## 1.1. Read SRA data

The `SraRunTable.txt` was downloaded from all **SRA Experiements** in NCBI for [BioProject PRJNA305824](https://www.ncbi.nlm.nih.gov/bioproject/PRJNA305824).

In [3]:
import pandas as pd

sra_df = pd.read_csv('SraRunTable.txt.gz')
print(f'Table rows: {len(sra_df)}')
sra_df.head(3)

Table rows: 65


,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,BioSampleModel,Bytes,Center Name,collected_by,...,Phagetype,Platform,ReleaseDate,Sample Name,Serovar,SRA Study,STRAIN,sub_species,Host_disease,Host
0,SRR3028736,WGS,419,621431550,PRJNA305824,SAMN04334627,Pathogen.cl,367138294,MCGILL UNIVERSITY,hospital,...,19,ILLUMINA,2015-12-19T00:00:00Z,SH12-001,Heidelberg,SRP067504,SH12-001,enterica,Salmonella gastroenteritis,Homo sapiens
1,SRR3028737,WGS,426,433789482,PRJNA305824,SAMN04334628,Pathogen.cl,251988185,MCGILL UNIVERSITY,hospital,...,19,ILLUMINA,2015-12-19T00:00:00Z,SH12-002,Heidelberg,SRP067504,SH12-002,enterica,Salmonella gastroenteritis,Homo sapiens
2,SRR3028738,WGS,432,407605751,PRJNA305824,SAMN04334629,Pathogen.cl,238018601,MCGILL UNIVERSITY,hospital,...,19,ILLUMINA,2015-12-19T00:00:00Z,SH12-003,Heidelberg,SRP067504,SH12-003,enterica,Salmonella gastroenteritis,Homo sapiens


## 1.2. Reduce to single SRA run per genomic sample

Some samples have multiple SRA runs. We pick the largest run in this case and reduce the table down to 59 samples/SRA runs.

In [4]:
df = sra_df.sort_values('Bases', ascending=False).groupby('Sample Name').agg(
    {'Sample Name': 'count', 'Run': 'first'}, axis='columns')
df[df['Sample Name'] > 1]

,Sample Name,Run
Sample Name,,
SH12-003,2,SRR3684173
SH12-007,2,SRR3684194
SH13-004,2,SRR3711286
SH13-006,2,SRR3711252
SH14-009,2,SRR3711296
SH14-028,2,SRR3712208


In [5]:
sra_df_reduced = sra_df.set_index('Run').loc[df['Run'].tolist()].reset_index().set_index('Sample Name')
print(f'Table rows: {len(sra_df_reduced)}')
sra_df_reduced.head(3)

Table rows: 59


,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,BioSampleModel,Bytes,Center Name,collected_by,...,PFGE_SecondaryEnzyme_pattern,Phagetype,Platform,ReleaseDate,Serovar,SRA Study,STRAIN,sub_species,Host_disease,Host
Sample Name,,,,,,,,,,,,,,,,,,,,,
SH08-001,SRR3028792,WGS,429,354123684,PRJNA305824,SAMN04334683,Pathogen.cl,197484364,MCGILL UNIVERSITY,hospital,...,SHBNI.0001,19,ILLUMINA,2015-12-19T00:00:00Z,Heidelberg,SRP067504,SH08-001,enterica,Salmonella gastroenteritis,Homo sapiens
SH09-29,SRR3028793,WGS,422,519366460,PRJNA305824,SAMN04334684,Pathogen.cl,288691068,MCGILL UNIVERSITY,hospital,...,SHBNI.0001,26,ILLUMINA,2015-12-19T00:00:00Z,Heidelberg,SRP067504,SH09-29,enterica,Salmonella gastroenteritis,Homo sapiens
SH10-001,SRR3028783,WGS,421,387145160,PRJNA305824,SAMN04334674,Pathogen.cl,233911529,MCGILL UNIVERSITY,hospital,...,SHBNI.0001,19,ILLUMINA,2015-12-19T00:00:00Z,Heidelberg,SRP067504,SH10-001,enterica,Salmonella gastroenteritis,Homo sapiens


## 1.3. Save as metadata

Save the table as `metadata.tsv`.

In [6]:
sra_df_reduced.to_csv('metadata.tsv', sep='\t', index=True)

In [8]:
!gzip -f metadata.tsv

# 2. Download SRA data

To download sra data, first we create the download command.

In [24]:
command_df = sra_df_reduced.reset_index()[['Sample Name', 'Run']]
command_df['command'] = command_df.apply(lambda x: f'fasterq-dump --threads 4 -O fastq -o {x["Sample Name"]} --split-files {x["Run"]}', axis='columns')
command_df.head(3)

,Sample Name,Run,command
0,SH08-001,SRR3028792,fasterq-dump --threads 4 -O fastq -o SH08-001 ...
1,SH09-29,SRR3028793,fasterq-dump --threads 4 -O fastq -o SH09-29 -...
2,SH10-001,SRR3028783,fasterq-dump --threads 4 -O fastq -o SH10-001 ...


Now write download command to a file.

In [29]:
command_df['command'].to_csv('commands.txt', index=False, header=False)

Now run these commands with parallel to schedule them all, downloading a few files at once

In [31]:
!conda run --name sra-tools parallel -j 2 -a commands.txt

spots read      : 824,262
reads read      : 1,648,524
reads written   : 1,648,524
spots read      : 1,230,549
reads read      : 2,461,098
reads written   : 2,461,098
spots read      : 919,490
reads read      : 1,838,980
reads written   : 1,838,980
spots read      : 797,406
reads read      : 1,594,812
reads written   : 1,594,812
spots read      : 963,726
reads read      : 1,927,452
reads written   : 1,927,452
spots read      : 1,390,852
reads read      : 2,781,704
reads written   : 2,781,704
spots read      : 1,505,500
reads read      : 3,011,000
reads written   : 3,011,000
spots read      : 799,810
reads read      : 1,599,620
reads written   : 1,599,620
spots read      : 1,482,963
reads read      : 2,965,926
reads written   : 2,965,926
spots read      : 1,016,422
reads read      : 2,032,844
reads written   : 2,032,844
spots read      : 1,300,892
reads read      : 2,601,784
reads written   : 2,601,784
spots read      : 2,115,600
reads read      : 4,231,200
reads written   : 4,231,200
sp

Now gzip all files

In [37]:
import glob

fastq_files = glob.glob('fastq/*.fastq')
!parallel -j 32 gzip ::: {' '.join(fastq_files)}

# 3. MLST analysis

MLST/cgMLST analysis currently needs assembled genomes and is not a part of the main pipeline. So first I must assembled the genomes.

## 3.1. Assemble genomes

In [1]:
import os
from pathlib import Path

assembly_dir = Path('assemblies')"
if not assembly_dir.exists():
    os.mkdir(assembly_dir)

In [24]:
assembly_command_df = sra_df_reduced.reset_index()[['Sample Name']]
assembly_command_df['command'] = assembly_command_df.apply(lambda x: f'skesa --cores 8 --reads '
    f'fastq/{x["Sample Name"]}_1.fastq.gz,fastq/{x["Sample Name"]}_2.fastq.gz --contigs_out assemblies/{x["Sample Name"]}.fasta', axis='columns')
assembly_command_df.head(3)

,Sample Name,command
0,SH08-001,skesa --cores 8 --reads fastq/SH08-001_1.fastq...
1,SH09-29,skesa --cores 8 --reads fastq/SH09-29_1.fastq....
2,SH10-001,skesa --cores 8 --reads fastq/SH10-001_1.fastq...


In [25]:
assembly_command_df['command'].to_csv('assembly-commands.txt', index=False, header=False)
# Remove quotes inserted by to_csv
!sed -i -e 's/"//g' assembly-commands.txt

In [26]:
# Ran below in command-line window since it would take a while in Jupyter
#!conda run --name skesa parallel -j 8 -a assembly-commands.txt

## 3.2. Regular MLST

In [39]:
!conda run --name mlst mlst --version
!conda run --name mlst mlst --nopath assemblies/*.fasta > all.mlst.tsv 2> /dev/null

mlst 2.19.0



In [46]:
!head -n 3 all.mlst.tsv
!wc -l all.mlst.tsv

SH08-001.fasta	senterica	15	aroC(2)	dnaN(7)	hemD(9)	hisD(9)	purE(5)	sucA(9)	thrA(12)
SH09-29.fasta	senterica	15	aroC(2)	dnaN(7)	hemD(9)	hisD(9)	purE(5)	sucA(9)	thrA(12)
SH10-001.fasta	senterica	15	aroC(2)	dnaN(7)	hemD(9)	hisD(9)	purE(5)	sucA(9)	thrA(12)
60 all.mlst.tsv


## 3.3. SISTR cgMLST

In [43]:
!conda run --name sistr sistr --version
!conda run --name sistr sistr -f csv -o all.sistr-predictions.tsv -p all.sistr-profiles.tsv --use-full-cgmlst-db \
    --qc -t 32 assemblies/*.fasta 

sistr_cmd 1.1.1



In [45]:
!head -n 3 all.sistr-profiles.tsv | cut -f 1,2,3,4 -d ','
!wc -l all.sistr-profiles.tsv

,NZ_AOXE01000059.1_60,NZ_AOXE01000019.1_14,NZ_AOXE01000085.1_58
SH08-001,3009296265,1560521952,3999802771
SH09-29,3009296265,1560521952,3999802771
60 all.sistr-profiles.tsv


# 3. Create index

In [47]:
!gdi --version

gdi, version 0.2.0.dev0


In [48]:
!gdi init salmonella-project

Initializing empty project in [salmonella-project]


## 3.1. Create index of variants and kmers

In [49]:
!gdi --project-dir salmonella-project --ncores 48 analysis \
    --reference-file reference/NC_011083.gbk.gz --use-conda \
    --kmer-size 31 --kmer-size 51 --kmer-size 71 --kmer-scaled 100 fastq/*.fastq.gz

2021-08-08 14:25:44 INFO: Automatically structuring 118 input files into assemblies/reads
2021-08-08 14:25:44 INFO: Processing 59 genomes to identify mutations
2021-08-08 14:25:44 INFO: Including snpeff annotations in snakemake results
2021-08-08 14:25:44 INFO: Running Snakemake for rule all
2021-08-08 15:03:07 INFO: Finished running snakemake.
2021-08-08 15:03:07 INFO: Indexing processed VCF files defined in [/home/CSCScience.ca/apetkau/workspace/genomics-data-index-examples/examples/create-index/snakemake-assemblies.1628450744.3240273/gdi-input.fofn]
2021-08-08 15:03:07 WARNING: Attempting to set global database_path_translator=<genomics_data_index.storage.model.db.DatabasePathTranslator.DatabasePathTranslator object at 0x7fef541f5f70> but it is already set
2021-08-08 15:03:07 INFO: Attempting to load reference genome=[reference/NC_011083.gbk.gz]
2021-08-08 15:03:10 INFO: Sample batch 1/1: Stage 1/2 (Insert): Processed 0% (0/59) samples
2021-08-08 15:03:13 INFO: Sample batch 1/1: Sta

## 3.2. Build ML tree from variants

In [50]:
!gdi --project-dir salmonella-project --ncores 48 rebuild tree \
    --align-type full --extra-params '--fast' NC_011083

2021-08-08 15:16:20 INFO: Started rebuilding tree for reference genome [NC_011083]
2021-08-08 15:20:19 INFO: Finished rebuilding tree


## 3.3. Insert regular MLST results

In [51]:
!gdi --project-dir salmonella-project load mlst-tseemann all.mlst.tsv

Loading MLST results from all.mlst.tsv
2021-08-08 15:20:22 INFO: Stage 1/2 (Insert): Processed 0% (0/59) samples
2021-08-08 15:20:22 INFO: Stage 1/2 (Insert): Processed 8% (5/59) samples
2021-08-08 15:20:22 INFO: Stage 1/2 (Insert): Processed 17% (10/59) samples
2021-08-08 15:20:22 INFO: Stage 1/2 (Insert): Processed 25% (15/59) samples
2021-08-08 15:20:22 INFO: Stage 1/2 (Insert): Processed 34% (20/59) samples
2021-08-08 15:20:22 INFO: Stage 1/2 (Insert): Processed 42% (25/59) samples
2021-08-08 15:20:22 INFO: Stage 1/2 (Insert): Processed 51% (30/59) samples
2021-08-08 15:20:22 INFO: Stage 1/2 (Insert): Processed 59% (35/59) samples
2021-08-08 15:20:22 INFO: Stage 1/2 (Insert): Processed 68% (40/59) samples
2021-08-08 15:20:22 INFO: Stage 1/2 (Insert): Processed 76% (45/59) samples
2021-08-08 15:20:22 INFO: Stage 1/2 (Insert): Processed 85% (50/59) samples
2021-08-08 15:20:22 INFO: Stage 1/2 (Insert): Processed 93% (55/59) samples
2021-08-08 15:20:22 INFO: Stage 1/2 (Insert): Process

## 3.4. Insert SISTR cgMLST

In [52]:
!gdi --project-dir salmonella-project load mlst-sistr all.sistr-profiles.tsv

Loading cgMLST results from all.sistr-profiles.tsv
2021-08-08 15:20:25 INFO: Stage 1/2 (Insert): Processed 0% (0/59) samples
2021-08-08 15:20:25 INFO: Stage 1/2 (Insert): Processed 8% (5/59) samples
2021-08-08 15:20:25 INFO: Stage 1/2 (Insert): Processed 17% (10/59) samples
2021-08-08 15:20:25 INFO: Stage 1/2 (Insert): Processed 25% (15/59) samples
2021-08-08 15:20:25 INFO: Stage 1/2 (Insert): Processed 34% (20/59) samples
2021-08-08 15:20:25 INFO: Stage 1/2 (Insert): Processed 42% (25/59) samples
2021-08-08 15:20:25 INFO: Stage 1/2 (Insert): Processed 51% (30/59) samples
2021-08-08 15:20:25 INFO: Stage 1/2 (Insert): Processed 59% (35/59) samples
2021-08-08 15:20:25 INFO: Stage 1/2 (Insert): Processed 68% (40/59) samples
2021-08-08 15:20:25 INFO: Stage 1/2 (Insert): Processed 76% (45/59) samples
2021-08-08 15:20:25 INFO: Stage 1/2 (Insert): Processed 85% (50/59) samples
2021-08-08 15:20:25 INFO: Stage 1/2 (Insert): Processed 93% (55/59) samples
2021-08-08 15:20:25 INFO: Stage 1/2 (Inse

# 4. Zip up index

In [53]:
!zip -r salmonella-project.zip salmonella-project 1> /dev/null